# Memory

In [ ]:
from dotenv import load_dotenv
import warnings
from langchain_openai import ChatOpenAI, OpenAI
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains import LLMChain

warnings.filterwarnings('ignore')
load_dotenv()
llm = OpenAI()
# chat = ChatOpenAI()
set_llm_cache(InMemoryCache())

## ChatMessageHistory

In [ ]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

In [ ]:
patient_dialogue1 = "Doctor, I have been feeling a bit under the weather lately."
sample_response1 = "Under the weather? Did you try checking the forecast before stepping out? You might need a weather app prescription!"

patient_dialogue2 = "My throat has been sore, and I have a cough."
sample_response2 = "The classic sore throat symphony! I recommend a strong dose of chicken soup and a dialy karaoke session. Sing it out, and your throat will thank you."

patient_dialogue3 = "I have a headache."
sample_response3 = "Headache, you say? Have you tried negotiating with it? Maybe it's just looking for a better job inside your brain!"

In [ ]:
history.add_user_message(patient_dialogue1)
history.add_ai_message(sample_response1)

history.add_user_message(patient_dialogue2)
history.add_ai_message(sample_response2)

history.add_user_message(patient_dialogue3)
history.add_ai_message(sample_response3)

In [ ]:
history

In [ ]:
history.messages

## ConversationBufferMemory

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

In [ ]:
response = conversation.invoke("Hi, how are you doing?")
response

In [ ]:
response = conversation.invoke("How is your health?")
response

In [ ]:
response = conversation.invoke("Tell me a fact about volcanoes?")
response

In [ ]:
response['history']

In [ ]:
conversation.memory.chat_memory

In [ ]:
conversation.memory.chat_memory = history

In [ ]:
conversation

In [ ]:
conversation.invoke("I have a stomach pain")

In [ ]:
# Deleting
# replacing
# Dumping
# Loading

In [ ]:
# Deleting & replacing
messages_history = conversation.memory.chat_memory.messages
messages_history

In [ ]:
conversation.memory.chat_memory.messages = messages_history[2:]

In [ ]:
conversation.memory.chat_memory

In [ ]:
# Dumping

from joblib import load, dump

memory_filepath = "../output/conversation_memory.pkl"

dump(conversation.memory, memory_filepath)

In [ ]:
loaded_memory = load(memory_filepath)

new_llm =  OpenAI()

loaded_conv = ConversationChain(llm=new_llm, memory=loaded_memory)

print(loaded_conv.memory.buffer)

## ConversationBufferWindowMemory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

memory = ConversationBufferWindowMemory(
    k=1, # Interaction: 1 Human message, 1 AI response
    human_prefix="Stark",
    ai_prefix="Jarvis"
)

conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

In [ ]:
conversation.invoke("Hi")

In [ ]:
conversation.invoke("Good to see you")

In [ ]:
conversation.invoke("Okay, bye")


## ConversationSummaryMemory

In [ ]:
from langchain.memory import ConversationSummaryMemory

In [ ]:
memory = ConversationSummaryMemory(llm=llm, max_token_limit=50)

coversation = ConversationChain(llm=llm, memory=memory, verbose=True)

In [ ]:
response = coversation.invoke("Plan a 10 day budget trip to Europe from India?")
response

In [ ]:
response = coversation.invoke("Which place should I visit on the first day?")
response

In [ ]:
response = coversation.invoke("Give me a detailed plan for my second day?")
response

In [ ]:
coversation.memory

In [ ]:
from langchain.chains import ConversationalRetrievalChain

In [ ]:
chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type='stuff',
    retriever=compression_retriever, # retriever
    return_source_documents=True,
    memory=memory
)